### 1. Introduction & Problem Statement

### 2. Data Exploration & Understanding

In [ ]:
import pandas as pd

# Getting the data
url = 'https://archive.ics.uci.edu/static/public/891/data.csv'
df = pd.read_csv(url)

# Set display options
pd.set_option('display.max_columns', None)

# 1. Basic Dataset Information
print("Dataset Shape:", df.shape)
print("\nFirst 5 rows:")
display(df.head())

# Checking data types and basic statistics
print("\nData Types:")
display(df.dtypes)

print("\nBasic statistics:")
display(df.describe())

# Checking for missing values
print("\nMissing values per column:")
display(df.isnull().sum())

# Checking column names
print("\nColumn names:")
print(df.columns.tolist())

# Checking unique values for categorical columns
print("\nSample of unique values in each column (first 5):")
for col in df.columns:
    if df[col].dtype == 'object' or df[col].nunique() < 10:
        print(f"{col}: {df[col].unique()[:5]}")

### 3. Data Cleaning & Preprocessing

In [ ]:
# balance class distribition by randomly removing excess rows with people without diabities
num_with_diabetes = df['Diabetes_binary'].value_counts()[1]  # Count of class 1 (people with diabetes)
df_no_diabetes = df[df['Diabetes_binary'] == 0].sample(n=num_with_diabetes, random_state=42)
df_with_diabetes = df[df['Diabetes_binary'] == 1]
df_balanced = pd.concat([df_no_diabetes, df_with_diabetes])
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

print(df_balanced['Diabetes_binary'].value_counts())

Diabetes_binary
1    35346
0    35346
Name: count, dtype: int64


### 4. Applying Differential Privacy Techniques

### 5. Feature Engineering & Selection

### 6. Model Selection & Training

### 7. Evaluation & Performance Metrics

### 8. Results & Discussion

### 9. Conclusion & Future Work